MLOpsへデプロイした、Hydrogen Torch Metric Learningモデルへのスコアリングの実施

In [57]:
import requests
import json

import pandas as pd
import numpy as np

MLOpsリクエストサンプル
```bash
curl -X POST -H "Content-Type: application/json" -d @- https://model.internal.dedicated.h2o.ai/d4e5a4ac-9ca0-4e60-af0e-0b175c1997a8/model/score << EOF
 {
   "fields": [
      "input"
   ],
   "rows": [
      [
         "text"
      ]
   ]
} 
EOF
```

複数オブザベーションのリクエストの場合
```bash
curl -X POST -H "Content-Type: application/json" -d @- https://model.internal.dedicated.h2o.ai/d4e5a4ac-9ca0-4e60-af0e-0b175c1997a8/model/score << EOF
 {
   "fields": ["input"],
   "rows": [
      ["text"], ["text2"]
   ]
} 
EOF
```

In [2]:
!ls data

jobdb_sample.csv


In [10]:
df = pd.read_csv('data/jobdb_sample.csv')
print(df.shape)
df

(10, 5)


,job_id,industry,role,rank,skill_sets
0,1,化学工業,安全管理者,シニア,- 長年の経験と知識を持った化学工業の安全管理に堪能すること\n- 法令や規制に基づいた安全...
1,3,農業,物流担当,ミドル,- 農業物流の知識と経験\n- 貨物管理と配送計画の作成\n- 仕入れと倉庫管理のスキル\n...
2,9,不動産業,コンサルタント,シニア,- 不動産市場の深い理解と経験\n- 不動産法令や政策の知識\n- 顧客対応やネゴシエーショ...
3,16,自動車製造業,製造オペレーター,ミドル,- 複雑な機械操作能力：製造オペレーターとしては、複雑な機械を操作する能力が必要です。\n-...
4,18,林業,営業担当,ミドル,- 林業分野の知識\n- 営業経験\n- 顧客対応能力\n- プレゼンテーション能力\n- ...
5,26,不動産業,資産管理担当,ミドル,- 不動産に関する知識と経験\n- 資産管理に堪能な計画力と実行力\n- 不動産仕訳帳の管理...
6,27,エネルギー業,リスクアナリスト,ジュニア,- Basic understanding of risk management princ...
7,29,医療・ヘルスケア業,医師,ジュニア,- 医学の基礎知識と技能\n- 患者の相談や診断に堪能なコミュニケーション能力\n- 医療法...
8,37,繊維業,生産管理,ミドル,- 繊維生産プロセスの知識と経験\n- 生産計画と調整能力\n- 品質管理の経験\n- 生産...
9,43,農業,農業エンジニア,ミドル,- 農業技術知識\n- 機械操作能力\n- データ分析スキル\n- 適切な耕作技術の知識\n...


In [14]:
print(df.loc[0, 'skill_sets'])

- 長年の経験と知識を持った化学工業の安全管理に堪能すること
- 法令や規制に基づいた安全管理計画の策定および実施
- 労働安全法、環境対策法、火災Prevencion法などの法令に堪能すること
- 労働者の安全訓練の計画、実施、評価の能力
- 労働災害の事例研究、分析、対策策定の能力
- 安全環境の継続的改善に努めること
- 労働者の安全意識向上に努めること
- 安全関連事業者との協力関係の整備および維持の能力
- 労働者の安全意識調査の計画、実施、評価の能力
- 災害時のリーダーシップの発揮


In [42]:
rows = [[s] for s in df['skill_sets']]
print(len(rows))
rows[:3]

10


[['- 長年の経験と知識を持った化学工業の安全管理に堪能すること\n- 法令や規制に基づいた安全管理計画の策定および実施\n- 労働安全法、環境対策法、火災Prevencion法などの法令に堪能すること\n- 労働者の安全訓練の計画、実施、評価の能力\n- 労働災害の事例研究、分析、対策策定の能力\n- 安全環境の継続的改善に努めること\n- 労働者の安全意識向上に努めること\n- 安全関連事業者との協力関係の整備および維持の能力\n- 労働者の安全意識調査の計画、実施、評価の能力\n- 災害時のリーダーシップの発揮'],
 ['- 農業物流の知識と経験\n- 貨物管理と配送計画の作成\n- 仕入れと倉庫管理のスキル\n- 購買と販売の知識\n- 財務管理と予算 planing\n- 人間関係のスキル\n- 計画と組織の能力\n- クライアント対応とサービス精度\n- IT技能（WMS, TMS, ERP）\n- 英語の基本的な会話能力'],
 ['- 不動産市場の深い理解と経験\n- 不動産法令や政策の知識\n- 顧客対応やネゴシエーション能力\n- 事業計画や予算管理のスキル\n- 不動産開発プロジェクトの計画と実行\n- リスク管理と戦略的考え方\n- 業界網を取り巻く関係の築き方\n- 経営戦略の立案と推進\n- 不動産資産の評価と調整\n- 企業コミュニケーションやプレゼンテーション能力']]

In [43]:
json_data = dict(fields=["input"], rows=rows)
json_data

{'fields': ['input'],
 'rows': [['- 長年の経験と知識を持った化学工業の安全管理に堪能すること\n- 法令や規制に基づいた安全管理計画の策定および実施\n- 労働安全法、環境対策法、火災Prevencion法などの法令に堪能すること\n- 労働者の安全訓練の計画、実施、評価の能力\n- 労働災害の事例研究、分析、対策策定の能力\n- 安全環境の継続的改善に努めること\n- 労働者の安全意識向上に努めること\n- 安全関連事業者との協力関係の整備および維持の能力\n- 労働者の安全意識調査の計画、実施、評価の能力\n- 災害時のリーダーシップの発揮'],
  ['- 農業物流の知識と経験\n- 貨物管理と配送計画の作成\n- 仕入れと倉庫管理のスキル\n- 購買と販売の知識\n- 財務管理と予算 planing\n- 人間関係のスキル\n- 計画と組織の能力\n- クライアント対応とサービス精度\n- IT技能（WMS, TMS, ERP）\n- 英語の基本的な会話能力'],
  ['- 不動産市場の深い理解と経験\n- 不動産法令や政策の知識\n- 顧客対応やネゴシエーション能力\n- 事業計画や予算管理のスキル\n- 不動産開発プロジェクトの計画と実行\n- リスク管理と戦略的考え方\n- 業界網を取り巻く関係の築き方\n- 経営戦略の立案と推進\n- 不動産資産の評価と調整\n- 企業コミュニケーションやプレゼンテーション能力'],
  ['- 複雑な機械操作能力：製造オペレーターとしては、複雑な機械を操作する能力が必要です。\n- 品質意識：製造業では、品質管理が非常に重要であるため、高い品質意識を持っていることが求められます。\n- 安全意識：安全上の理由から、常に安全操作を意識することが求められます。\n- 生産性向上の意識：生産性を上げることで労力を削減し、費用を節約することが重要であるため、生産性向上の意識を持っていることが求められます。\n- チームワーク能力：製造業では、多くの場合チームワークが必要であるため、チームワーク能力を持っていることが求められます。\n- 計画的思考能力：製造業では、常に計画的に行動することが重要であるため、計画的思考能力を持っていることが求められます。\n- 継続的学習意欲：技術は進歩し

In [44]:
response = requests.post(url='https://model.internal.dedicated.h2o.ai/d4e5a4ac-9ca0-4e60-af0e-0b175c1997a8/model/score', 
                         headers={'content-type': 'application/json'}, 
                         data=json.dumps(json_data))
response

<Response [200]>

In [45]:
response.json().keys()

dict_keys(['id', 'fields', 'score'])

In [46]:
response.json()['id']

'cf7ac268-043f-4d25-ae0a-34876b1976c0'

In [47]:
response.json()['fields']

['output']

In [52]:
res = json.loads(response.json()['score'][0][0])
res.keys()

dict_keys(['embeddings', 'cosine_similarities', 'similar_texts'])

In [66]:
print(type(res['embeddings']))
np.array(res['embeddings']).shape

# 各オブザーベーションの256次元ベクトル

<class 'list'>


(10, 256)

In [63]:
print(type(res['cosine_similarities']))
np.array(res['cosine_similarities']).shape

# 各オブザーベーションの上位類似度

<class 'list'>


(10, 10)

In [72]:
print(type(res['similar_texts']))
np.array(res['similar_texts']).shape

# 各オブザーベーションの類似が上位のオブザーベーション番号

<class 'list'>


(10, 10)

In [71]:
# オブザーベーション0に対する類似上位のオブザーベーション番号と、類似度
res['similar_texts'][0], res['cosine_similarities'][0]

([0, 3, 7, 9, 5, 8, 2, 6, 4, 1],
 [1.0000001192092896,
  0.47952544689178467,
  0.47258275747299194,
  0.42199546098709106,
  0.23024627566337585,
  0.1796545386314392,
  0.09474027156829834,
  0.07530471682548523,
  0.0,
  0.0])

In [73]:
# コサイン類似度の計算確認

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [76]:
cos_sim(res['embeddings'][0], res['embeddings'][0])

0.9999999999999998

In [77]:
cos_sim(res['embeddings'][0], res['embeddings'][3])

0.47952548594556316

In [78]:
cos_sim(res['embeddings'][0], res['embeddings'][7])

0.4725827409980136

In [83]:
# オブザーベーション0に対するその他のオブザーベーション間との類似度
[cos_sim(res['embeddings'][0], i) for i in res['embeddings']]

[0.9999999999999998,
 -0.044221069318673166,
 0.09474025937944765,
 0.47952548594556316,
 -0.13586338860603503,
 0.23024626356168726,
 0.07530470831608085,
 0.4725827409980136,
 0.17965452589983175,
 0.42199543138073586]